In [1]:
import numpy as np
import pandas as pd
import import_ipynb
from utils import show

np.random.seed(12)

importing Jupyter notebook from utils.ipynb


In [2]:
def get_lst_range(data):
    iszero = np.concatenate(([0],
                             np.greater(data,0).view(np.int8),
                            [0]))
    absdiff = np.abs(np.diff(iszero))
    lst_range = np.where(absdiff==1)[0].reshape(-1,2)
    return lst_range

In [3]:
def lst_final(data):
    shifted_data_zero = np.concatenate((data[1:],[0]))
    zero_range = np.divide(shifted_data_zero, data)
    zero_range = np.concatenate(([data.iloc[0]], zero_range))-1
    zero_range = zero_range[:-1]
    iszero = np.concatenate(([0],
                             np.greater(zero_range,0).view(np.int8),
                            [0]))
    absdiff = np.abs(np.diff(iszero))
    lst_range = np.where(absdiff==1)[0].reshape(-1,2)
    return lst_range

In [4]:
def individual_return(data):
    shifted_data_zero = np.concatenate((data[1:],[0]))
    zero_range = np.divide(shifted_data_zero, data)
    zero_range = np.concatenate(([data.iloc[0]], zero_range))-1
    zero_range = zero_range[:-1]
    return zero_range

In [5]:
def get_lst_return(df,range_df):
    len_range_df = range_df.diff(axis=1).stack()[::2]
    lst_mask = len_range_df == np.max(len_range_df,axis=0)
    lst_df = range_df[lst_mask]
    
    result_ls = []
    for fdname in lst_mask.columns:
        single_df = pd.DataFrame(columns=['START','END','LEN','RETURN'])
        for i in lst_df[fdname].dropna().index:
            start = lst_df[fdname].iloc[i]['START'].astype(int)
            pre_start = start - 1 if start != 0 else None
            end = lst_df[fdname].iloc[i]['END'].astype(int)
            real_end = end - 1
            #print(type(start))
            #print(df[fdname].iloc[start-1])
            if pre_start:
                value = df[fdname].iloc[real_end] / df[fdname].iloc[pre_start] -1
            else:
                value = df[fdname].iloc[real_end] - 1
            single_ser = pd.Series({'START':df.index[start],
                                    'END':df.index[real_end],
                                    'LEN':end - start,
                                    'RETURN':value})
            single_df = single_df.append(single_ser,ignore_index=True)
            single_df = single_df[single_df['RETURN']==single_df['RETURN'].max()]
        single_df.columns = [[fdname,fdname,fdname,fdname],
                             ['START','END','LEN','RETURN']]
        result_ls.append(single_df)
    result_df = pd.concat(result_ls,axis=1)
    if result_df.isnull().any().any():
        result_df = result_df.fillna(method='ffill').dropna(how='any', axis=0)
    return result_df

In [6]:
df = pd.DataFrame(np.random.uniform(0.,2.5,(18,5)),columns=list('ABCDE'))
df.index = [20140531,20140630,20140731,20140831,20140930,20141031,20141130,20141231,20150131,20150228,
                20150331,20150430,20150531,20150630,20150731,20150831,20150930,20151030]
df


,A,B,C,D,E
20140531,0.385407,1.850124,0.658288,1.334348,0.036437
20140630,2.296868,2.251787,0.083554,2.392373,0.343023
20140731,0.709571,1.515208,2.360563,2.131839,0.005648
20140831,1.303065,1.380094,1.213444,1.920335,0.401792
20140930,1.911401,0.052024,0.338025,0.290683,0.774744
20141031,1.678632,1.178074,2.040421,0.723967,1.832815
20141130,1.756556,0.818924,0.836619,2.445145,1.561455
20141231,2.375784,1.918689,2.062523,1.016601,1.128271
20150131,1.001579,2.487845,0.443910,2.406492,1.048126
20150228,1.060131,1.157872,0.934308,1.163770,0.087921


In [7]:
cumprod_df = df.cumprod()

In [8]:
indi_return = cumprod_df.apply(individual_return, axis=0)

In [9]:
start = 20150507
end = 20151107

In [10]:
freq_prev_idx = np.array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 2, 3, 4, 5])
freq_prev_idx

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  1,  2,  3,  4,
        5])

In [11]:
actual_start = df.index[df.index.searchsorted(start, side='right') - 1]


In [12]:
ranges = pd.Series(range(int(start <= actual_start), freq_prev_idx.max() + 2)).apply(
    lambda x: (x, freq_prev_idx.searchsorted(x, side='left') - 1))

In [13]:
ranges

0    (0, 11)
1    (1, 12)
2    (2, 13)
3    (3, 14)
4    (4, 15)
5    (5, 16)
6    (6, 17)
dtype: object

In [23]:
r = ranges[0]
_data = indi_return.iloc[r[0]:r[1]+1]
_data.index.names = ['date']
_data

,A,B,C,D,E
date,,,,,
20140531,-0.614593,0.850124,-0.341712,0.334348,-0.963563
20140630,1.296868,1.251787,-0.916446,1.392373,-0.656977
20140731,-0.290429,0.515208,1.360563,1.131839,-0.994352
20140831,0.303065,0.380094,0.213444,0.920335,-0.598208
20140930,0.911401,-0.947976,-0.661975,-0.709317,-0.225256
20141031,0.678632,0.178074,1.040421,-0.276033,0.832815
20141130,0.756556,-0.181076,-0.163381,1.445145,0.561455
20141231,1.375784,0.918689,1.062523,0.016601,0.128271
20150131,0.001579,1.487845,-0.556090,1.406492,0.048126


In [30]:
_data.apply(lambda x:pd.Series(get_lst_range(x),index=np.arange(_data.shape[0])),axis=0,reduce=False)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: The reduce argument is deprecated and will be removed in a future version. You can specify result_type='reduce' to try to reduce the result to the original dimensions
  if __name__ == '__main__':


ValueError: ('Length of passed values is 2, index implies 12', 'occurred at index A')

In [ ]:
result_ls = []

for offset in range(len(ranges)):
    r = ranges[offset]
    range_lst_tmp = []
    # fdname = tst_df.columns[fdidx]
    for fdidx in range(df.columns.size):
        fdname = df.columns[fdidx]
        _data = indi_return.iloc[r[0]:r[1] + 1, fdidx]
        single_rst = get_lst_range(_data) + offset
        range_lst_tmp.append(pd.DataFrame(single_rst, columns=[[fdname, fdname], ['START', 'END']]))
    range_df = pd.concat(range_lst_tmp, axis=1)
    range_df.columns.names = ['FUND', 'RANGE']
    range_df.index.names = ['idx']
    result_single_df = get_lst_return(cumprod_df, range_df)
    result_ls.append(result_single_df)
result_df = pd.concat(result_ls, axis=0)
newindex = df.index[df.index.searchsorted(start, side='right') - 1:]
result_df.index = newindex

In [ ]:
result_df

In [ ]:
result_df.columns.levels[0]

In [ ]:
ror_ls = []
for fdname in df.columns:
    _fd = result_df[fdname]
    _fd['ROR'] = np.power(1 + _fd['RETURN'],
                          12 / _fd['LEN']) - 1
    _fd.columns=pd.MultiIndex.from_product([[fdname],
                                            ['START',
                                        'END','LEN','RETURN','ROR']])
    ror_ls.append(_fd)
    
ror_df = pd.concat(ror_ls,axis=1)
ror_df